In [1]:
import pandas as pd

In [2]:
import re
import string

In [3]:
from nltk.corpus import stopwords
import nltk

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

In [5]:
from sklearn.decomposition import NMF

In [6]:
df_union = pd.read_csv('df_union.csv')  

In [7]:
df_union

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_postal-code,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,61B3197983D553C6CCC12D3C265E4375,Mexico,19.0,18.0,30.0,28.0,AlejandraMexico,"May 10, 2009",May 2009,29721005.0,...,78258,TX,125 North Loop 1604 West,NaN,3.0,Homewood Suites by Hilton Hotel San Antonio North,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...
9996,9CE843F2646875FCB4EE774CAB6FCB68,"Detroit, Michigan",31.0,25.0,55.0,22.0,mjwingfans,"July 3, 2010",May 2010,69543852.0,...,46250,IN,6817 E. 82nd Street,NaN,2.5,Hampton Inn Indianapolis - NE / Castleton,NaN,37209.0,hotel,http://www.tripadvisor.com/Hotel_Review-g37209...
9997,7BFD165584BDE13CBD73A9AF1C1B2BF7,Austin,9.0,11.0,17.0,9.0,AustinBarb,"December 30, 2011",December 2011,122265474.0,...,78216,TX,850 Halm Boulevard,NaN,2.5,La Quinta Inn & Suites San Antonio Airport,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...
9998,7B5194085DEE03D4FCC5589C6353D683,near Liège,5.0,6.0,5.0,5.0,elpelso,"March 5, 2008",March 2008,14063603.0,...,77003,TX,1810 Bell Ave,NaN,2.5,Holiday Inn Express Houston-Downtown Conventio...,NaN,56003.0,hotel,http://www.tripadvisor.com/Hotel_Review-g56003...


## Text preprocessing

In [8]:
df_union.columns

Index(['author_id', 'author_location', 'author_num_cities',
       'author_num_helpful_votes', 'author_num_reviews',
       'author_num_type_reviews', 'author_username', 'date', 'date_stayed',
       'id', 'num_helpful_votes', 'offering_id', 'ratings_check_in_front_desk',
       'ratings_cleanliness', 'ratings_location', 'ratings_overall',
       'ratings_rooms', 'ratings_service', 'ratings_value', 'text', 'title',
       'via_mobile', 'ratings_sleep_quality',
       'ratings_business_service_(e_g_internet_access)', 'address_locality',
       'address_postal-code', 'address_region', 'address_street-address',
       'details', 'hotel_class', 'name', 'phone', 'region_id', 'type', 'url'],
      dtype='object')

Delete reviews that contains more non-english words than english, because a lot of french words defined as english

In [9]:
words = set(nltk.corpus.words.words())

In [10]:
df_union.text[df_union.text == '0']

Series([], Name: text, dtype: object)

In [11]:

words = set(nltk.corpus.words.words())

sent = "Io andiamo to the beach with my amico."
" ".join(w for w in nltk.wordpunct_tokenize(sent) \
         if w.lower() in words or not w.isalpha())
# 'Io to the beach with my'

'Io to the beach with my .'

In [12]:
df_union.address_locality.value_counts()

New York City    3141
San Francisco    1026
Chicago           728
San Diego         654
Los Angeles       603
Washington DC     560
Boston            508
Seattle           382
San Antonio       330
Philadelphia      264
Dallas            209
Houston           200
Denver            198
Austin            182
Phoenix           165
Baltimore         130
Indianapolis      123
Charlotte         107
Memphis           103
Jacksonville      102
Columbus           79
San Jose           62
Fort Worth         58
Detroit            48
El Paso            38
Name: address_locality, dtype: int64

In [13]:
# Compound Term Extraction
repl = lambda x: x.replace('new york', 'new_york').replace('nyc', 'new_york').replace('times square', 'times_square').replace('new york city', 'new_york')#.replace('nyc', 'new_york').replace('nyc', 'new_york').replace('nyc', 'new_york').replace('nyc', 'new_york')

# Text preprocessing steps - remove numbers, captial letters and punctuation
alphanumeric = lambda x: re.sub('\w*\d\w*', ' ', x)
punc_lower = lambda x: re.sub('[%s]' % re.escape(string.punctuation), ' ', x.lower())

words = set(nltk.corpus.words.words())
english_only = lambda x: " ".join(w for w in nltk.wordpunct_tokenize(x) \
         if w.lower() in words or not w.isalpha())

df_union['text'] = df_union.text.map(alphanumeric).map(punc_lower).map(repl)#.map(english_only) #.map(repl)
df_union['title'] = df_union.title.map(alphanumeric).map(punc_lower).map(repl)#.map(english_only) #.map(repl)
df_union.head()

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_postal-code,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,19107,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...


In [14]:
df_union['Is_English'] = 1

In [15]:
# check if review written in English
# all review that contain at least a half of English words mark as english, otherway - not

for i, row in enumerate(df_union.text):
    len_review = len(row)
    english_review = " ".join(w for w in nltk.wordpunct_tokenize(row) \
         if w.lower() in words or not w.isalpha())
    
    len_english = len(english_review)
    
    if len_review > len_english * 2:
        df_union.Is_English[i] = 0
    else:
        df_union.Is_English[i] = 1

<ipython-input-15-58d0da8f527a>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_union.Is_English[i] = 1
<ipython-input-15-58d0da8f527a>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_union.Is_English[i] = 0


In [16]:
df_union.text[df_union.Is_English == 0]

9       middenin downtown la een hotel van grote klass...
13      positivo \ngrandioso hall con unos techos espe...
16      hotel extrement bien situé idéal pour pouvoir ...
17      dopo aver fatto una ricerca approfondita degli...
20      ci siamo trovati benissimo  camere confortevol...
                              ...                        
9936    j ai voulu essayé cet hôtel   étoiles pour une...
9940    das motel liegt verkehrsgünstig um die stadt p...
9952    comme son nom l indique  l hôtel est en plein ...
9987    hotellet ligger sentralt i san francisco  ikke...
9995    definitivamente recomendaría este hotel   los ...
Name: text, Length: 1231, dtype: object

In [119]:
df_english = df_union[(df_union['Is_English'] == 1)]

In [18]:
df_english 

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url,Is_English
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,611A2AACA2267A12D25AFF59523DBC04,Columbia Maryland,10.0,26.0,11.0,10.0,LoveLife82,"October 29, 2008",October 2008,21389392.0,...,TX,2838 Cinema Ridge,NaN,3.0,Residence Inn San Antonio Seaworld,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...,1
9996,9CE843F2646875FCB4EE774CAB6FCB68,"Detroit, Michigan",31.0,25.0,55.0,22.0,mjwingfans,"July 3, 2010",May 2010,69543852.0,...,IN,6817 E. 82nd Street,NaN,2.5,Hampton Inn Indianapolis - NE / Castleton,NaN,37209.0,hotel,http://www.tripadvisor.com/Hotel_Review-g37209...,1
9997,7BFD165584BDE13CBD73A9AF1C1B2BF7,Austin,9.0,11.0,17.0,9.0,AustinBarb,"December 30, 2011",December 2011,122265474.0,...,TX,850 Halm Boulevard,NaN,2.5,La Quinta Inn & Suites San Antonio Airport,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...,1
9998,7B5194085DEE03D4FCC5589C6353D683,near Liège,5.0,6.0,5.0,5.0,elpelso,"March 5, 2008",March 2008,14063603.0,...,TX,1810 Bell Ave,NaN,2.5,Holiday Inn Express Houston-Downtown Conventio...,NaN,56003.0,hotel,http://www.tripadvisor.com/Hotel_Review-g56003...,1


In [19]:
# removing stop words from the text
stop = stopwords.words('english')
stop += ['.', ',', '(', ')', "'", '"']
stop += ['hotel', 'stay']
stop = set(stop)

#### Delete reviews where author_id or id id None

In [120]:
df_english.dropna(subset = ["author_id"], inplace=True)

<ipython-input-120-d3a8d7b68583>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english.dropna(subset = ["author_id"], inplace=True)


In [121]:
df_english.dropna(subset = ["id"], inplace=True)

<ipython-input-121-2558c79a1070>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_english.dropna(subset = ["id"], inplace=True)


In [122]:
df_english.reset_index(drop=True,inplace=True)

In [123]:
df_english.head(10)

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url,Is_English
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
5,E99894E1BBD9F2D5F1E5E5F7F31ED36B,"Saskatoon, Saskatchewan, Canada",20.0,97.0,24.0,24.0,saskatoonguy,"August 10, 2007",August 2007,8389979.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
6,D13B22BC4C7A1DC2AD877A39452EEC70,"New York City, New York",11.0,19.0,14.0,13.0,GRRRLTRAVELER,"January 24, 2009",January 2009,23874828.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
7,F41398D3661814E5973AB287490FDE5B,"London, United Kingdom",12.0,65.0,24.0,12.0,Holidays-R-Good,"March 19, 2005",NaN,3287965.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
8,25F3E453887C8931BFDC476D5649FB66,GAINESVILLE,12.0,14.0,15.0,15.0,RobertGainesville,"January 31, 2011",December 2010,95126489.0,...,CA,506 South Grand Avenue,NaN,3.5,Millennium Biltmore Hotel Los Angeles,NaN,32655.0,hotel,http://www.tripadvisor.com/Hotel_Review-g32655...,1
9,C54D22AC653BDA8CCAD7502159B54816,"Thíra, Kikladhes, Greece",13.0,9.0,34.0,3.0,Christo G,"November 30, 2012",November 2012,146459488.0,...,CA,506 South Grand Avenue,NaN,3.5,Millennium Biltmore Hotel Los Angeles,NaN,32655.0,hotel,http://www.tripadvisor.com/Hotel_Review-g32655...,1


## Document-Term Matrix

In [20]:
# The first document-term matrix has default Count Vectorizer values - counts of unigrams

vectorizer = CountVectorizer(stop_words = stop)

doc_word = vectorizer.fit_transform(df_english.text)

pd.DataFrame(doc_word.toarray(), columns=vectorizer.get_feature_names()).head()

,aa,aaa,aaaaaaaaaaaaaaaaaaaaaaaaaa,aaaawwwwesome,aapointed,aargh,aaron,aarp,aback,abandoned,...,zombies,zone,zones,zoning,zoo,zoomed,zooming,zoribel,zorro,zuli
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Topic modeling

### Non-Negative Matrix Factorization (NMF)

In [21]:
nmf_model = NMF(10)
doc_topic_nmf = nmf_model.fit_transform(doc_word)
doc_topic_nmf.shape

(8769, 10)

In [22]:
topic_word_nmf = nmf_model.components_
topic_word_nmf.shape

(10, 22492)

In [23]:
words_nmf = vectorizer.get_feature_names()
t = nmf_model.components_.argsort(axis=1)[:,-1:-8:-1]
topic_words_nmf = [[words_nmf[e] for e in l] for l in t]
topic_words_nmf

[['room', 'bed', 'floor', 'bathroom', 'view', 'small', 'king'],
 ['one', 'get', 'night', 'like', 'time', 'bed', 'also'],
 ['us', 'back', 'day', 'went', 'get', 'time', 'night'],
 ['great', 'location', 'service', 'place', 'really', 'time', 'view'],
 ['staff', 'friendly', 'helpful', 'clean', 'stayed', 'comfortable', 'well'],
 ['rooms', 'two', 'clean', 'beds', 'small', 'location', 'floor'],
 ['nice', 'really', 'area', 'also', 'pool', 'lobby', 'bathroom'],
 ['desk', 'front', 'service', 'told', 'called', 'check', 'said'],
 ['would', 'recommend', 'could', 'night', 'really', 'think', 'definitely'],
 ['good', 'breakfast', 'free', 'location', 'service', 'clean', 'area']]

### Try using TF-IDF instead of Count Vectorizer

In [24]:
# Create TF-IDF versions of the Count Vectorizers created earlier in the exercise
from sklearn.feature_extraction.text import TfidfVectorizer

In [25]:
tfidf = TfidfVectorizer(stop_words=stop)

In [26]:
doc_word_tfidf = tfidf.fit_transform(df_english.text)              

In [27]:
pd.DataFrame(doc_word_tfidf.toarray(), columns=tfidf.get_feature_names()).head()

,aa,aaa,aaaaaaaaaaaaaaaaaaaaaaaaaa,aaaawwwwesome,aapointed,aargh,aaron,aarp,aback,abandoned,...,zombies,zone,zones,zoning,zoo,zoomed,zooming,zoribel,zorro,zuli
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### NMF - tf-ifd

In [28]:
nmf_model_2 = NMF(10)
doc_topic_nmf_2 = nmf_model_2.fit_transform(doc_word_tfidf)
doc_topic_nmf_2.shape

(8769, 10)

In [29]:
topic_word_nmf_2 = nmf_model_2.components_
topic_word_nmf_2.shape

(10, 22492)

In [30]:
words_2 = tfidf.get_feature_names()
t_2 = nmf_model_2.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words_nmf_2 = [[words_2[e] for e in l] for l in t_2]
topic_words_nmf_2

[['room', 'bed', 'bathroom', 'floor', 'small', 'night'],
 ['staff', 'friendly', 'helpful', 'clean', 'comfortable', 'would'],
 ['us', 'desk', 'front', 'get', 'told', 'check'],
 ['new_york', 'times_square', 'central', 'subway', 'park', 'city'],
 ['walking', 'walk', 'distance', 'parking', 'restaurants', 'within'],
 ['great', 'location', 'place', 'really', 'loved', 'chicago'],
 ['good', 'rooms', 'value', 'location', 'price', 'bit'],
 ['service', 'excellent', 'restaurant', 'food', 'wonderful', 'beautiful'],
 ['free', 'breakfast', 'airport', 'shuttle', 'internet', 'coffee'],
 ['nice', 'really', 'rooms', 'pool', 'bar', 'lobby']]

In [31]:
doc_topic_nmf_2

array([[0.02607795, 0.00525559, 0.04295018, ..., 0.02077725, 0.        ,
        0.        ],
       [0.01286431, 0.01830322, 0.        , ..., 0.01953761, 0.        ,
        0.0374795 ],
       [0.0184583 , 0.05069648, 0.02051456, ..., 0.00114014, 0.        ,
        0.        ],
       ...,
       [0.034891  , 0.01671629, 0.02990179, ..., 0.        , 0.05564342,
        0.        ],
       [0.00184419, 0.        , 0.02247876, ..., 0.        , 0.        ,
        0.05658918],
       [0.0062499 , 0.        , 0.07199305, ..., 0.01191433, 0.0024537 ,
        0.        ]])

In [32]:
H = pd.DataFrame(doc_topic_nmf_2.round(5),
                 index = df_english.text
                )
H

,0,1,2,3,4,5,6,7,8,9
text,,,,,,,,,,
i recently stayed here for two nights while in town for a conference check in was smooth and took about minutes i was surprised to learn that the clerk gave me a room with a view of city hall well what that meant was you have a room that borders a very busy street i felt like i was trying to sleep at a nascar event if you stay here make sure you get a room that faces away from city hall \ni received my receipt under the door in the morning and it had a food charge for which was not mine i brought it to the attention of the checkout desk and i was told that they would need to research it out with the restaurant manager i guess guilty until proven innocent they have subsequently removed it from my bill i still do not appreciate the inconvenience \nthe hotel over all is beautiful and the rooms are spacious,0.02608,0.00526,0.04295,0.00000,0.02469,0.00000,0.00258,0.02078,0.00000,0.00000
i stayed here for a business trip and stayed extra nights room was very clean had a nice view i will def stay here when i visit again the restaurant downstairs was good it was close to the convention center and other areas for walking the city i was able to walk to independence hall no problem,0.01286,0.01830,0.00000,0.00429,0.07816,0.00000,0.04217,0.01954,0.00000,0.03748
after a disappointing night at the hampton inn i moved my family here what a difference the room was spacious and comfortable the staff was suburb and super friendly \nthe only problem we had was with a chamber maid who refused to not enter the room eventhough the baby was sleeping management to their credit was very helpful and apologetic this is a place i would definitely recommend \nthis is one of theee marriott properties next to each other the others being the marriott downtown and the residence inn we were there for a conference at the marriott downtown but were able to stay at the courtyard for about less if you go out the back door the hotels are literally across the street from each other,0.01846,0.05070,0.02051,0.00000,0.03495,0.00030,0.00000,0.00114,0.00000,0.00000
we stayed at the marriott courtyard in downtown philadelphia for one night in august \nthe hotel is across the street from city hall and about a block from reading terminal market the hotel is also right next door to the marriott residence inn and right behind the large marriott the hotel has a landmark facade and lobby so it appears quite elegant the staff was very helpful and alert the rooms are nice but nothing super special old tvs marginal pillows \nthe indoor pool is nice for a quick cool down and the fitness center is very well equipped,0.01437,0.01947,0.00000,0.00000,0.05862,0.00000,0.00000,0.00650,0.01076,0.05895
the hotels exterior blends in well with the awe inspiring city hall right across the street the interior was recently don t know when upgraded and has that newer hotel feel our room was clean and comfortable and looked like a basic courtyard hotel room we ate sandwiches at the hotels restaraunt wich was good but not great the atmosphere was relaxing and inviting wich was a needed retreat after beeing on the busy streets of philly best tips don t use mapquest to find this hotel it s not correct use the valet parking it s a steep a day but finding good parking can be difficult you will have to walk alot or take the philly flash to get around,0.01433,0.00210,0.00799,0.00000,0.05178,0.00949,0.03613,0.00741,0.00307,0.00000
...,...,...,...,...,...,...,...,...,...,...
i will be a residence inn patron for life it was by far the best hotel i have ever stayed at normally i leave reviews only if hotels are less than mediocre or absolutely worth it if its in between i don t bother i have stayed at many in between hotels for both business and pleasure and i gotta say this hotel was by far the best one ever \nbrand new modern decor clean smells new great kitchen space huge we had a and a both were huge \nthe breakfast was amazing hot foods with great selection i think i 

### !!! try different topic modeling on tf-idf

### LSA

In [33]:
from sklearn.decomposition import TruncatedSVD

In [34]:
# Acronynms: Latent Semantic Analysis (LSA) is just another name for 
#  Signular Value Decomposition (SVD) applied to Natural Language Processing (NLP)
lsa = TruncatedSVD(10)
doc_topic_lsa = lsa.fit_transform(doc_word_tfidf)
lsa.explained_variance_ratio_

array([0.00352777, 0.00560414, 0.00384405, 0.00329267, 0.00315416,
       0.00302141, 0.00287006, 0.00260617, 0.00254137, 0.00242305])

In [35]:
doc_topic_lsa

array([[ 0.19549326, -0.09450028,  0.03049956, ..., -0.05287361,
        -0.01884262, -0.04432511],
       [ 0.21723229,  0.08494309, -0.11272163, ..., -0.07169662,
        -0.04147925,  0.00942172],
       [ 0.17650512, -0.00599924,  0.0143426 , ...,  0.04632059,
        -0.0429084 , -0.08199364],
       ...,
       [ 0.22622609, -0.04327085,  0.01255547, ..., -0.00259425,
         0.09613957, -0.07224738],
       [ 0.13520774, -0.02584541, -0.06216302, ...,  0.05328773,
        -0.04849439, -0.01231811],
       [ 0.15850018, -0.13847605,  0.0835778 , ...,  0.01305825,
         0.01661515, -0.06731715]])

In [36]:
Vt = pd.DataFrame(doc_topic_lsa.round(5),
             index = df_english.text)
Vt

,0,1,2,3,4,5,6,7,8,9
text,,,,,,,,,,
i recently stayed here for two nights while in town for a conference check in was smooth and took about minutes i was surprised to learn that the clerk gave me a room with a view of city hall well what that meant was you have a room that borders a very busy street i felt like i was trying to sleep at a nascar event if you stay here make sure you get a room that faces away from city hall \ni received my receipt under the door in the morning and it had a food charge for which was not mine i brought it to the attention of the checkout desk and i was told that they would need to research it out with the restaurant manager i guess guilty until proven innocent they have subsequently removed it from my bill i still do not appreciate the inconvenience \nthe hotel over all is beautiful and the rooms are spacious,0.19549,-0.09450,0.03050,0.00193,-0.02020,-0.01835,-0.01113,-0.05287,-0.01884,-0.04433
i stayed here for a business trip and stayed extra nights room was very clean had a nice view i will def stay here when i visit again the restaurant downstairs was good it was close to the convention center and other areas for walking the city i was able to walk to independence hall no problem,0.21723,0.08494,-0.11272,0.01304,-0.04435,-0.04263,-0.01140,-0.07170,-0.04148,0.00942
after a disappointing night at the hampton inn i moved my family here what a difference the room was spacious and comfortable the staff was suburb and super friendly \nthe only problem we had was with a chamber maid who refused to not enter the room eventhough the baby was sleeping management to their credit was very helpful and apologetic this is a place i would definitely recommend \nthis is one of theee marriott properties next to each other the others being the marriott downtown and the residence inn we were there for a conference at the marriott downtown but were able to stay at the courtyard for about less if you go out the back door the hotels are literally across the street from each other,0.17651,-0.00600,0.01434,-0.02530,-0.00004,-0.06909,-0.03694,0.04632,-0.04291,-0.08199
we stayed at the marriott courtyard in downtown philadelphia for one night in august \nthe hotel is across the street from city hall and about a block from reading terminal market the hotel is also right next door to the marriott residence inn and right behind the large marriott the hotel has a landmark facade and lobby so it appears quite elegant the staff was very helpful and alert the rooms are nice but nothing super special old tvs marginal pillows \nthe indoor pool is nice for a quick cool down and the fitness center is very well equipped,0.18777,0.02614,-0.09379,-0.04995,-0.05007,-0.02190,-0.02288,0.04633,-0.10570,0.00031
the hotels exterior blends in well with the awe inspiring city hall right across the street the interior was recently don t know when upgraded and has that newer hotel feel our room was clean and comfortable and looked like a basic courtyard hotel room we ate sandwiches at the hotels restaraunt wich was good but not great the atmosphere was relaxing and inviting wich was a needed retreat after beeing on the busy streets of philly best tips don t use mapquest to find this hotel it s not correct use the valet parking it s a steep a day but finding good parking can be difficult you will have to walk alot or take the philly flash to get around,0.16292,-0.00017,-0.06599,-0.02131,-0.06216,0.01076,0.00626,-0.01684,0.00885,-0.03649
...,...,...,...,...,...,...,...,...,...,...
i will be a residence inn patron for life it was by far the best hotel i have ever stayed at normally i leave reviews only if hotels are less than mediocre or absolutely worth it if its in between i don t bother i have stayed at many in between hotels for both business and pleasure and i gotta say this hotel was by far the best one ever \nbrand new modern decor clean smells new great kitchen space huge we had a and a both were huge \nthe breakfast was amazing hot foods

In [37]:
words_lsa = tfidf.get_feature_names()
t_lsa = lsa.components_.argsort(axis=1)[:,-1:-7:-1]
topic_words_lsa = [[words_lsa[e] for e in l] for l in t_lsa]
topic_words_lsa

[['room', 'great', 'staff', 'good', 'nice', 'would'],
 ['great', 'location', 'staff', 'friendly', 'clean', 'helpful'],
 ['great', 'us', 'service', 'staff', 'wonderful', 'time'],
 ['new_york', 'times_square', 'room', 'central', 'subway', 'city'],
 ['room', 'clean', 'good', 'staff', 'bed', 'friendly'],
 ['great', 'service', 'room', 'location', 'bar', 'nice'],
 ['good', 'rooms', 'new_york', 'service', 'hotels', 'really'],
 ['nice', 'rooms', 'place', 'would', 'clean', 'helpful'],
 ['good', 'great', 'breakfast', 'us', 'free', 'location'],
 ['nice', 'us', 'breakfast', 'really', 'good', 'bar']]

### LDA

### Text Recommendation through Matrix Factorization

In [39]:
from sklearn.metrics import pairwise_distances

In [40]:
pairwise_distances(doc_topic_nmf_2[4].reshape(1,-1),doc_topic_nmf_2,metric='cosine')

array([[0.43470495, 0.11474472, 0.47462099, ..., 0.76647155, 0.4487776 ,
        0.84389475]])

In [42]:
pairwise_distances(doc_topic_nmf_2[4].reshape(1,-1),doc_topic_nmf_2,metric='cosine').argsort()

array([[   4, 7064, 2620, ..., 8728, 4824, 2047]])

##### You can now take any text and make a recommendation for it:

In [43]:
t = ['basketball is the best']

In [44]:
vt = vectorizer.transform(t)

In [45]:
tt = nmf_model.transform(vt)

In [47]:
pairwise_distances(tt,doc_topic_nmf_2,metric='cosine').argsort()

array([[1272, 7307, 4651, ..., 3881, 1086, 7583]])

In [48]:
df_english.text[1272]

'i m currently staying in the sofitel chicago water tower for the third time in the last three months  i m someone who is fortunate enough to travel fairly frequently in the u s  and i go to europe twice a year  i get to stay in some in very nice places  but this hotel is probably my favorite  i guess i like this place because it s rather upscale  but not in a stuffy way  \nthe building itself is one of the major positive aspects of staying here  it is a beautiful  contemporary high rise  the rooms i ve stayed in have always been in immaculate condition and have very nice  spacious marble bathrooms  this is a hotel that may not be right for everyone    the design aesthetic is very modern  it also doesn t seem to offer rooms with twin beds  the rooms seem to all have king size beds  but if having two beds isn t a requirement and you don t want traditional decor  this place is hard to beat  \neverything about the physical facility of this hotel is really very good  i noticed on this stay

Simple recommender: list of similar hotels

In [51]:
from collections import defaultdict

In [109]:
user_movie_map = defaultdict(list)
hotel_user_map = defaultdict(list)

In [90]:
user_movie_map

defaultdict(list, {})

In [56]:
df_english.columns

Index(['author_id', 'author_location', 'author_num_cities',
       'author_num_helpful_votes', 'author_num_reviews',
       'author_num_type_reviews', 'author_username', 'date', 'date_stayed',
       'id', 'num_helpful_votes', 'offering_id', 'ratings_check_in_front_desk',
       'ratings_cleanliness', 'ratings_location', 'ratings_overall',
       'ratings_rooms', 'ratings_service', 'ratings_value', 'text', 'title',
       'via_mobile', 'ratings_sleep_quality',
       'ratings_business_service_(e_g_internet_access)', 'address_locality',
       'address_postal-code', 'address_region', 'address_street-address',
       'details', 'hotel_class', 'name', 'phone', 'region_id', 'type', 'url',
       'Is_English'],
      dtype='object')

In [58]:
df_english

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url,Is_English
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9994,611A2AACA2267A12D25AFF59523DBC04,Columbia Maryland,10.0,26.0,11.0,10.0,LoveLife82,"October 29, 2008",October 2008,21389392.0,...,TX,2838 Cinema Ridge,NaN,3.0,Residence Inn San Antonio Seaworld,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...,1
9996,9CE843F2646875FCB4EE774CAB6FCB68,"Detroit, Michigan",31.0,25.0,55.0,22.0,mjwingfans,"July 3, 2010",May 2010,69543852.0,...,IN,6817 E. 82nd Street,NaN,2.5,Hampton Inn Indianapolis - NE / Castleton,NaN,37209.0,hotel,http://www.tripadvisor.com/Hotel_Review-g37209...,1
9997,7BFD165584BDE13CBD73A9AF1C1B2BF7,Austin,9.0,11.0,17.0,9.0,AustinBarb,"December 30, 2011",December 2011,122265474.0,...,TX,850 Halm Boulevard,NaN,2.5,La Quinta Inn & Suites San Antonio Airport,NaN,60956.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60956...,1
9998,7B5194085DEE03D4FCC5589C6353D683,near Liège,5.0,6.0,5.0,5.0,elpelso,"March 5, 2008",March 2008,14063603.0,...,TX,1810 Bell Ave,NaN,2.5,Holiday Inn Express Houston-Downtown Conventio...,NaN,56003.0,hotel,http://www.tripadvisor.com/Hotel_Review-g56003...,1


In [69]:
len(df_english)

8769

In [78]:
df_english.loc[0, 'author_id']

'2E4442B5FF4C485F4EF4FD49B66AD171'

In [93]:
user_movie_map

defaultdict(list,
            {'2E4442B5FF4C485F4EF4FD49B66AD171': [17784086.0, 17784086.0],
             '556A4A70DCA457DE95FD95B9CE217014': [136233716.0, 136233716.0],
             '8D6ECB2EB034591EF86E540319D24992': [6161247.0, 6161247.0],
             'DDCEA80A1BFE32D8C91EFE473737A229': [37482297.0, 37482297.0],
             '19568376B8C1295886B69B322E4C6055': [31138794.0, 31138794.0],
             'E99894E1BBD9F2D5F1E5E5F7F31ED36B': [8389979.0, 8389979.0],
             'D13B22BC4C7A1DC2AD877A39452EEC70': [23874828.0, 23874828.0],
             'F41398D3661814E5973AB287490FDE5B': [3287965.0, 3287965.0],
             '25F3E453887C8931BFDC476D5649FB66': [95126489.0, 95126489.0]})

In [96]:
df_english.shape[0]

8769

In [112]:
df_english.head(10)

,author_id,author_location,author_num_cities,author_num_helpful_votes,author_num_reviews,author_num_type_reviews,author_username,date,date_stayed,id,...,address_region,address_street-address,details,hotel_class,name,phone,region_id,type,url,Is_English
0,2E4442B5FF4C485F4EF4FD49B66AD171,"Raleigh, NC",10.0,32.0,16.0,14.0,shaniz1,"July 11, 2008",July 2008,17784086.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
1,556A4A70DCA457DE95FD95B9CE217014,"Nashville, TN",5.0,2.0,12.0,5.0,Grumpylicous,"August 4, 2012",September 2011,136233716.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
2,8D6ECB2EB034591EF86E540319D24992,New York,13.0,55.0,21.0,20.0,AA-Platinum,"November 21, 2006",November 2006,6161247.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
3,DDCEA80A1BFE32D8C91EFE473737A229,New York,34.0,74.0,50.0,14.0,DavidTrip,"August 12, 2009",August 2009,37482297.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
4,19568376B8C1295886B69B322E4C6055,"Harrisburg, Pennsylvania",10.0,3.0,11.0,7.0,unclesamspad2,"May 30, 2009",May 2009,31138794.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
5,E99894E1BBD9F2D5F1E5E5F7F31ED36B,"Saskatoon, Saskatchewan, Canada",20.0,97.0,24.0,24.0,saskatoonguy,"August 10, 2007",August 2007,8389979.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
6,D13B22BC4C7A1DC2AD877A39452EEC70,"New York City, New York",11.0,19.0,14.0,13.0,GRRRLTRAVELER,"January 24, 2009",January 2009,23874828.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
7,F41398D3661814E5973AB287490FDE5B,"London, United Kingdom",12.0,65.0,24.0,12.0,Holidays-R-Good,"March 19, 2005",NaN,3287965.0,...,PA,21 N Juniper Street,NaN,3.0,Courtyard Philadelphia Downtown,NaN,60795.0,hotel,http://www.tripadvisor.com/Hotel_Review-g60795...,1
8,25F3E453887C8931BFDC476D5649FB66,GAINESVILLE,12.0,14.0,15.0,15.0,RobertGainesville,"January 31, 2011",December 2010,95126489.0,...,CA,506 South Grand Avenue,NaN,3.5,Millennium Biltmore Hotel Los Angeles,NaN,32655.0,hotel,http://www.tripadvisor.com/Hotel_Review-g32655...,1
11,C54D22AC653BDA8CCAD7502159B54816,"Thíra, Kikladhes, Greece",13.0,9.0,34.0,3.0,Christo G,"November 30, 2012",November 2012,146459488.0,...,CA,506 South Grand Avenue,NaN,3.5,Millennium Biltmore Hotel Los Angeles,NaN,32655.0,hotel,http://www.tripadvisor.com/Hotel_Review-g32655...,1


In [124]:
df_english.author_id[9]

'C54D22AC653BDA8CCAD7502159B54816'

In [125]:
for i in range(0, df_english.shape[0]):
#     print(df_english.author_id[i])
#     print(df_english.id[i])
#     print(i)""
    user_movie_map[df_english.author_id[i]].append(df_english.id[i])
    
    hotel_user_map[df_english.id[i]].append(df_english.author_id[i])

In [126]:
user_movie_map

defaultdict(list,
            {'2E4442B5FF4C485F4EF4FD49B66AD171': [17784086.0, 17784086.0],
             '556A4A70DCA457DE95FD95B9CE217014': [136233716.0, 136233716.0],
             '8D6ECB2EB034591EF86E540319D24992': [6161247.0, 6161247.0],
             'DDCEA80A1BFE32D8C91EFE473737A229': [37482297.0, 37482297.0],
             '19568376B8C1295886B69B322E4C6055': [31138794.0, 31138794.0],
             'E99894E1BBD9F2D5F1E5E5F7F31ED36B': [8389979.0, 8389979.0],
             'D13B22BC4C7A1DC2AD877A39452EEC70': [23874828.0, 23874828.0],
             'F41398D3661814E5973AB287490FDE5B': [3287965.0, 3287965.0],
             '25F3E453887C8931BFDC476D5649FB66': [95126489.0, 95126489.0],
             'C54D22AC653BDA8CCAD7502159B54816': [146459488.0],
             '18633BE1D9B2D22B837B44D9725AF2F9': [141361765.0],
             '66D7A24B171BE5E0FC9058FDC3A817FE': [133212732.0],
             '95DD9AFE73FDA704251346FBA56EFD88': [144660502.0],
             'C81752118872A24E25191F4C0E4A3C46': [10808

In [127]:
def get_similar_hotels(user_movie_map,hotel_user_map,m):
    biglist = []
    for u in hotel_user_map[m]: # get all users that liked this movie
        biglist.extend(user_movie_map[u]) # find all other movies those users liked and add to biglist
    return Counter(biglist).most_common(4)[1:] # use counter to 'count' the other movies that show up most common

In [131]:
from collections import Counter

In [133]:
get_similar_hotels(user_movie_map,hotel_user_map,132337022.0)

[(129780686.0, 1)]